In [1]:
#加载训练数据
import  pandas as pd
import pickle
import gensim
import numpy as np
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
CLS = 'age'
NUM_CLASSES = 10
from keras.layers import *
from keras.models import Model
import keras.backend as K
from  keras.utils import  multi_gpu_model
import keras
from keras.preprocessing.sequence import pad_sequences




train = pd.read_csv('../data/train/new_train.csv')
train_data = train[['init_text','top_key_text']]
label = train[CLS]

x_train, x_test, y_train, y_test = train_test_split(train_data,label.values,test_size=0.1)

#加载字典
with open('../model/other/muti-count-encode.pickle', 'rb') as f:
    muti_count_encode = pickle.load(f)

vocab=muti_count_encode.vocabulary_



#生成onthot稀疏编码
init_text_train = x_train['init_text'].values
onthoht_sparse_train = muti_count_encode.transform(init_text_train)

init_text_test = x_test['init_text'].values
onthoht_sparse_test = muti_count_encode.transform(init_text_test)

ONEHOT_LEN = onthoht_sparse_train.shape[1]



#将text转换为编码
topk_text_train = x_train['top_key_text'].values
topk_text_train = [i.split(',') for i in topk_text_train]
def transformer_code(vocab,t):
    return [vocab[word] for word in t if word !='']

topk_text_train = [transformer_code(vocab,i) for i in topk_text_train]

topk_text_test = x_test['top_key_text'].values
topk_text_test = [i.split(',') for i in topk_text_test]
topk_text_test = [transformer_code(vocab,i) for i in topk_text_test]


#生成word2vec编码
embedding = np.random.rand(len(vocab)+1,100)
def transfomer(x):
    return K.cast(x,'float32')


def get_model(embedding_weight,vocab,numclass,onehot_len):

    #wide
    one_hot_input = Input(shape=(onehot_len,),dtype= 'int32')


    #deep
    main_input = Input(shape=(200,), dtype='float32')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(len(vocab) + 1, 100, input_length=200, trainable=True,weights=[embedding_weight])
    embed = embedder(main_input)
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=48)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=48)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=48)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    cnn_drop = Dropout(0.5)(flat)

    #联合
    one_hot_input_f = Lambda(transfomer)(one_hot_input)
    concat = concatenate([one_hot_input_f, cnn_drop])
    main_output = Dense(numclass, activation='softmax')(concat)
    model = Model(inputs=[one_hot_input,main_input], outputs=main_output)

    return model


def batch_generator(onthoht_sparse,text,label,batch_size,numclass):

    while(True):
        idx = np.arange(len(text))
        np.random.shuffle(idx)
        X1,X2,Y=[],[],[]
        np.random.shuffle(idx)
        for i in idx:
            x_onehot = onthoht_sparse[i, :].toarray()[0]
            x_text = text[i][:200]
            y = keras.utils.to_categorical(label[i]-1, num_classes=numclass)
            X1.append(x_onehot)
            X2.append(x_text)
            Y.append(y)
            if len(X1) == batch_size or i == idx[-1]:
                X1 = np.array(X1)
                X2 =pad_sequences(X2, maxlen=200,value=len(vocab))
                yield [X1, X2], [Y]
                X1 = []
                X2 = []
                Y = []

Using TensorFlow backend.
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/tensorfl

In [2]:
class ParallelModelCheckpoint(ModelCheckpoint):
    def __init__(self,model,filepath, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1):
        self.single_model = model
        super(ParallelModelCheckpoint,self).__init__(filepath, monitor, verbose,save_best_only, save_weights_only,mode, period)

    def set_model(self, model):
        super(ParallelModelCheckpoint,self).set_model(self.single_model)

In [3]:
train_model = get_model(embedding,vocab,NUM_CLASSES,ONEHOT_LEN)
train_model.load_weights('../model/'+CLS+'/05_26_weights.best.hdf5')

parallel_model = multi_gpu_model(train_model, gpus=2)
parallel_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
file_path='../model/'+CLS+'/05_27_weights.best.hdf5'
checkpoint= ParallelModelCheckpoint(train_model,filepath=file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only = True)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode='max')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, onthoht_sparse,text,label,batch_size,numclass, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.label = label
        self.onthoht_sparse = onthoht_sparse
        self.text = text
        self.n_classes = numclass
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.text) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]



        # Generate data
        X1, X2, Y = self.__data_generation(indexs)
        
        return [X1,X2],[Y]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.text))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self,indexs):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        X1, X2, Y = [], [], []


        # Generate data
        for i in indexs:
            # Store sample
            x_onehot = self.onthoht_sparse[i, :].toarray()[0]
            x_text = self.text[i][:200]


            # Store class
            y = keras.utils.to_categorical(self.label[i] - 1, num_classes=self.n_classes)
            X1.append(x_onehot)
            X2.append(x_text)
            Y.append(y)
        X1 = np.array(X1)
        X2 = pad_sequences(X2, maxlen=200, value=len(vocab))


        return X1, X2, Y


In [5]:
trainD = DataGenerator(onthoht_sparse_train,topk_text_train,y_train,batch_size=10,numclass=10)
testD = DataGenerator(onthoht_sparse_test,topk_text_test,y_test,batch_size=10,numclass=10,shuffle = True)

In [ ]:
parallel_model.fit_generator(trainD,
                            callbacks=[early_stopping,checkpoint],
                            validation_steps=len(topk_text_test),
                            epochs=10,
                            validation_data=testD,
                            use_multiprocessing=False,
                            workers=20
                          )

Instructions for updating:
Use tf.cast instead.


/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 653741000 elements. This may consume a large amount of memory.
  num_elements)


Epoch 1/10
   90/81000 [..............................] - ETA: 21:20:24 - loss: 1.3212 - accuracy: 0.5111